## Word2Vec with SGD (Stochastic Gradient Descent) and NEG (Negative Sampling) 
### Implementation Steps:
1. Assign the target word and neighboring context words as **Positive** examples.
2. Assign randomly sampled words in the lexicon based on a unigram distrubution (built using word frequency) as **Negative** examples.
3. Train the model using a Logistic Classifier by optimizing the loss function.
4. Use the regression weights as the embedding vectors.

## Import Required Modules

In [9]:
import pandas as pd
import numpy as np
import pickle

import random
import string
import time

from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

import re
from collections import defaultdict

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rojin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
def sigmoid(x):
    '''Function to compute the value of x after applying the Sigmoid function'''
    return 1.0 /(1 + np.exp(-x))

## Data Preprocessing

In [8]:
def preprocessing(corpus):
    '''Function for data preprocessing'''
    processed = []
    
    stop_words = stopwords.words('english')
    
    # Split text corpus into sentences
    sentences = corpus.split(".")
    
    # Loop through each sentence
    for i in range(len(sentences)):
        
        # Remove leading and trailing characters
        sentences[i] = sentences[i].strip()
        
        # Split sentence into list of words
        sentence = sentences[i].split()
        
        # Remove punctuations
        x = [word.strip(string.punctuation) for word in sentence if word not in stop_words]
        
        # Convert to lower case
        x = [word.lower() for word in x]
        
        processed.append(x) 
        
    print('\nProcessed sentence is:',  processed)
        
    return processed

## Word2Vec with NEG

In [18]:
class Word2Vec_with_NEG:
    '''Implmentation of Skip-Gram Word2Vec model with negative sampling'''
    def __init__(self, sentences):
        self.sentences = sentences
        self.N = 5 # dimension of word embeddings
        self.learning_rate = 0.01 # learning rate
        self.epochs = 5000 # number of training epochs
        self.window = 2 # window size
        self.negative_rate = 5 #ratio of negative samples over positive samples
        self.min_count = 5 # minimum count of words to be considered
        self.word2idx = None
        self.unigram = None
        pass
    
    def generate_training_data(unigram_power=0.75):
        '''Function to generate the word counts and mapping from word to index and vice versa
        Input: List of tokenized sentences
        Output: 
        v: Vocabulary size
        word_list: list of words in vocabulary sorted in alphabetical order
        word2idx: dict with word as key and index as value
        word_freq: dict with word as key and frequency as value'''
        
         # Initialize a dictionary of word frequency
        word_freq = {}
        
        # Iterate over each sentence in the list of sentences
        for sent in self.sentences:
            # Iterate over each word in sentence
            for word in sent:
                # Create the frequency dictionary to count each word
                word_freq[word] = word_freq.get(word, 0) + 1

        # Remove words that have frequency < minCount
        if self.min_count > 1:
            word_freq = {word:freq for word, freq in word_freq.items() if freq >= self.min_count}

        # Create word2idx and idx2word dictionaries from word_list
        self.word2idx = {w: idx for (idx, w) in enumerate(word_freq.keys())}

        # Compute unigram
        
        # Initialize an array of unigram
        unigram = np.zeros(len(self.word2idx))
        
        # Iterate over list of words and calculate the probability for each word
        for word, frequency in word_freq.items():
            # Raise each word frequency to the power chosen
            f = frequency ** unigram_power
            # Update unigram array
            unigram[self.word2idx[word]] = f
        
        # Normalization
        self.unigram = unigram / np.sum(unigram)
    
    def generate_positive_words():
        '''Function to generate positive training words'''
        
        P = [] # Initialize list of positive words
        V = len(self.word2id) # Size of vocabulary
        
        N_sentences = len(self.sentences)
        
        # If the word does not exist in the dictionary (due to min_count) then set its index to -1
        sentences_index_form = [None]* N_Sentences
        for idx, sent in enumerate(self.sentences):
            sentences_index_form[idx] = [self.word2idx.get(w, -1) for w in sent]
        
        # For efficiency, pre-compute the number of positives for each word
        
        N_P = np.zeros(V, dtype=int) # Number of positive words for each word
        
        for idx, sent_word_indices in enumerate(sentences_index_form):
            for i, word_idx in enumerate(sent_word_indices):
                if word_idx < 0:
                    continue
                first = max(0, i-self.winSize)
                last = min(i+self.winSize+1, len(sent_word_indices))
                N_P[word_idx] += (last - first - 1)
                
                
        # Allocate the memory for P in advance
        P = [None]*V
        for word_idx in range(V):
            P[word_idx] = np.zeros(N_P[word_idx], dtype=int)
        P_next_position = [0]*V
        
        # Loop over each sentence in the corpus to extract the target word and context words
        for idx, sent_word_indices in enumerate(sentences_index_form):
            if (idx + 1) % 100 == 0:
                print('Processing sentence', idx + 1, '/', N_Sentences)
                
            # Iterate over each word in sentence and add to its Positive list
            for i, word_idx in enumerate(sent_word_indices):
                if word_idx < 0:
                    continue
                first = max(0, i-self.N)
                last = min(i+self.N+1, len(sent_word_indices))
                number_of_words = (last - first - 1)
                position = P_next_position[word_idx]
                P[word_idx][position:position+number_of_words] = np.asarray(sent_word_indices[first:i] + sent_word_indices[i+1:last])
                P_next_position[word_idx] += number_of_words
                
        if self.minCount > 1:
            for word_idx in range(V):
                P[word_idx] = np.delete(P[word_idx], np.where(P[word_idx] < 0))
   
        # Remove duplicates
        for word_idx in range(V):
            P[word_idx] = np.unique(P[word_idx])
        return P
    
    def negative_sampling(t, P):
        '''Function to draw negative samples from unigram distribution
        Input: 
        t: Target word
        P: List of positive words
        Output:
        List of indexes of negative samples
        '''
        # Remove indices of t and P as they cannot be negative
        invalid_indices = P.tolist() + [t]
        
        probabilities = np.copy(self.unigram)
        
         # To avoid mistakenly obtaining postive samples or t itself,set the probabilities of these indices to 0
        probabilities[invalid_indices] = 0
        
        probabilities /= np.sum(probabilities)
        negative_samples = np.random.choice(len(self.unigram), size=self.negativeRate, p=probabilities)

        return negative_samples
    
    def train(self, learning_rate, epochs):
        '''Function to train the model
        Output: Trained embeddings'''
        
        print('Generate training data')
        self.generate_training_data()

        print('\nGenerate positive words')
        P = self.generate_positive_words()

        V = len(self.word2idx)
        print('\nTotal number of words in Vocabulary =', V)

        # Initialization
        W = np.random.rand(self.N, V)
        C = np.random.rand(V, self.N)

        losses = []
        
        # Loop through each epoch
        for Pass in range(epochs):
            print('\nEpoch: ', Pass + 1)
            
            #Initialize loss
            loss = 0
            
            # For each target word in V
            for t in range(V):
                # Get the current embedding vectors
                wt = W[:, t]
                positive_samples = P[t]
                
                for p in positive_samples:
                    # Get the negative samples
                    negative_samples = self.negative_sampling(t, positive_samples)
                    
                    # Get the context vector
                    cp = C[p, :]
                    cn = C[negative_samples, :]
                    
                    # Get intermedite values
                    sp = sigmoid(-np.dot(cp, wt))
                    sn = sigmoid(np.dot(cn, wt))
                    
                    # Calculate the partial derivatives
                    dwt = - sp*cp + np.dot(sn, cn)
                    dcp = - sp*wt
                    dcn = np.outer(sn, wt)
                    
                    # Update the gradient descent
                    wt -= learning_rate*dwt
                    cp -= learning_rate*dcp
                    cn -= learning_rate*dcn
                    
                    # Calculate loss
                    loss += -np.log(sigmoid(np.dot(cp, wt))) + np.sum(-np.log(sigmoid(-np.dot(C_neg, wt))))
                    
                # Output progress
                if (t+1)%100 == 0:
                    print('\t step ' + str(t + 1) + '/' + str(V) + 'loss: %.2f'%(loss), end='\r')
            
            losses.append(loss)
            
            print('\tLoss: %.2f' %loss, 'Elapsed time:', time.time() - start, '(s)--------')

In [19]:
# Set random seed
np.random.seed(0) 

# Get text data
text = "Welcome students to the Department of Computer Science. We have great faculty and professors. We will have a welcome program today."

# Pre-process the data
corpus = preprocessing(text)


Processed sentence is: [['welcome', 'students', 'department', 'computer', 'science'], ['we', 'great', 'faculty', 'professors'], ['we', 'welcome', 'program', 'today'], []]
